# explanable AI (XAI)
딥러닝 모델은 수 많은 매개변수(parameter)와 비선형 연산으로 이루어져 있으며, 이는 본질적으로 블랙박스(Black Box)입니다. 즉, 모델의 내부의 동작 원리를 알기 어려운 구조입니다.

이 때문에 실제 현장에서 딥러닝 모델을 사용할 때, 그 결과를 신뢰할 수 있을지에 대한 의문이 생깁니다.

설명 가능한 인공지능(XAI)은 이러한 문제를 해결하고자 모델의 내부 작동 방식과 결과를 인간이 이해하고 검증할 수 있도록 도와줍니다. 이를 통해 모델의 예측 결과에 대한 신뢰성을 높이고, 모델 개선 방향을 제시하는 데 유용합니다.

그중에서도 시각화(visualization)를 통한 설명 방법은 인간이 모델의 내부 동작 원리를 보다 쉽게 이해하고 검증할 수 있도록 도와줍니다.

# Class activation Map(CAM)
클래스 활성화 맵(Class activation Map)은 [2016년 논문](http://cnnlocalization.csail.mit.edu/Zhou_Learning_Deep_Features_CVPR_2016_paper.pdf)에서 제안된 개념으로, CNN 모델이 특정 클래스(class)로 분류하는것에 기여한 입력 이미지의 영역을 시각화 하는 방법입니다.

일반적으로 CNN 모델은 마지막 Convolution 레이어 뒤에 Global average pooling (GAP) 레이어를 배치하여 feature map의 공간적 평균을 계산합니다. 

<img src="resources/GAP.png" style="width:400px">


| Layer | output shape |
|------|----------|
| input | 3x224x224  | 
|Conv|512x7x7 | 
|GAP|512x1x1|  
| fc |1000 |   



GAP 레이어의 주요 장점은 다음과 같습니다:
 - Translational Invariance: 물체가 이미지 내에서 어디에 위치하는지보다는 물체가 존재하는지 여부를 판단할 수 있도록 도와줍니다.
 - 차원 축소(Dimensionality Reduction): fc레이어 필요한 파라미터 수를 크게 줄여 오버피팅을 방지합니다. feature들의 공간적 평균을 사용해 정보 손실을 최소화하면서도 높은 분류 성능을 유지합니다
 - 해석 가능성 (Interpretable Features): GAP 레이어의 출력 벡터는 feature map의 공간적 평균을 나타내며, fc레이어의 가중치(weight) 값의 크기를 통해 각 feature들이 모델의 예측에 얼마나 기여하였는지(중요도)를 파악할 수 있습니다.

<img src="resources/CAM.png" style="width:800px">

Class activation Map (CAM) 아래 수식에 따라 계산됩니다.

마지막 conv layer의 공간적 위치 $(x, y)$에서 $k$번째 채널(activation map)의 출력을 $f_k(x, y)$라 하겠습니다.

이때, k번째 activation map의 global average pooling $F_k$은 아래와 같이 계산됩니다:
$$F_k = \sum_{x, y} f_k(x, y)$$

class $c$와 $F_k$에 대응하는 fc레이어의 가중치를 $w_k^c$라 할때,
class $c$의 fc레이어 출력값(softmax전 logit값) $S_c$는 다음과 같이 계산됩니다:
$$S_c = \sum_k w_k^c F_k$$

이 수식에서 알 수 있듯이, $w_k^c$는 class $c$ 판단에 대한 $F_k$값들의 중요도(영향)를 표현합니다.

$F_k = \sum_{x, y} f_k(x, y)$를 위 수식에 대입하면 아래와 같은 결과를 얻습니다.

$$S_c = \sum_k w_k^c  \sum_{x, y} f_k(x, y) = \sum_{x, y} \sum_k w_k^c  f_k(x, y)$$

여기서 우리는 **class activation map** $M_c$를 다음과 같이 정의합니다:
$$M_c(x, y) = \sum_k w_k^c f_k(x, y)$$

따라서 클래스 $c$에 대응하는 logit 값 $S_c$는 아래와 같이 표현되며, $M_c(x, y)$는 class $c$ 분류를 위한 각 공간 위치 $(x,y)$에서의 활성화 값들의 중요도를 나타냅니다.
$$S_c = \sum_{x, y} M_c(x, y)$$






In [ ]:
import torch
from torchvision import models, transforms

from PIL import Image

from utils import visualize_heatmap
from ImageNet_utils import clsidx_to_labels as imagenet_idx_to_labels

사전학습된 ResNet18을 이용하여 CAM을 실습해보겠습니다.

In [ ]:
def load_pretrained_model():
    model = models.resnet18(weights = "IMAGENET1K_V1")
    model.eval()
    return model

def preprocess_image(image_path):
    """
    Preprocess the input image for ResNet-50.

    Args:
        image_path (str): Path to the input image.

    Returns:
        torch.Tensor: Preprocessed image tensor.
    """
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],   # ImageNet means
            std=[0.229, 0.224, 0.225]     # ImageNet stds
        )
    ])
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0) #add batch dim
    return image_tensor

In [ ]:
model = load_pretrained_model()
image = preprocess_image("resources/airplane.jpg")
output = model(image)
class_idx = int(output.argmax())
print(f"image.shape: {image.shape}, class index : {class_idx}, class label : {imagenet_idx_to_labels[class_idx]}")

## Hook

CAM 구현을 위해서는 모델의 중간 출력값을 가져와야 합니다.

이를 위해 PyTorch hook 기능을 이용합니다. hook이란 `nn.Module` 중간에 원하는 코드를 삽입할 수 있게 해주는 기능입니다.

모듈에 적용되는 hook에는 총 3가지 종류가 있으며, 호출 순서는 다음과 같습니다:
forward_pre_hook → `forward()` → forward_hook → `backward()` → full_backward_hook

아래 예시는 `forward_hook`을 사용해 각 레이어의 중간 출력값을 저장하는 방법을 보여줍니다.

In [ ]:
model = load_pretrained_model()

intermediate_outputs = {}
handles = []

def get_activation(name):
    def hook(model, input, output):
        intermediate_outputs[name] = output.detach()
    return hook

for name, module in model.named_modules():
    handle = module.register_forward_hook(get_activation(name))
    handles.append(handle)

model(image) #Forward pass

for layer_name, output in intermediate_outputs.items():
    print(f"{layer_name} : {output.shape}")

또한, hook 함수에서 output을 수정하여 출력값을 변경할 수도 있습니다 (예: gradient clipping).

hook은 아래와 같이 `hook_handle`을 사용하여 삭제할 수 있습니다.

In [ ]:
for handle in handles:
    handle.remove()

## ResNet18


resnet18은 아래의 구조로 이루어져 있습니다:
- stem(convl, bn1, relu, maxpool)
- 네 개의 스테이지(layer1-4)
- avgpool
- fc

In [ ]:
print(model)

## <mark>실습</mark> Class Activation Map (CAM)
Class Activation Map 을 계산하는 함수 `compute_cam`을 완성하세요.
1. `forward_hook`을 이용하여 ResNet-18 모델의 `layer4`의 출력값(feature)을 가져옵니다.
2. 모델에 입력을 전달하여 forward pass를 수행합니다.
3. `hook_handle`을 통해 forward hook을 삭제합니다.
4. 모델의 `fc`레이어에서 `weight`값을 가져와, `target_class`에 해당하는 가중치를 얻습니다.
5. 위 수식을 참고하여 CAM 값을 계산합니다.

In [ ]:
def compute_cam(model, image_tensor, target_class):
    """
    Compute the Class Activation Map (CAM) heatmap for the target class.

    Args:
        model (torch.nn.Module): Pretrained model.
        image_tensor (torch.Tensor): Preprocessed image tensor.
        target_class (int): Target class index.

    Returns:
        torch.Tensor: CAM heatmap.
    """
    
    # Hook the feature extractor
    features = []
    def hook_feature(module, input, output):
        features.append(output.detach())
    hook_handle = model.layer4.register_forward_hook(hook_feature)

    # Forward pass
    output = model(image_tensor)

    hook_handle.remove()

    fc_weights = model.fc.weight  # (1000, 512)
    feature_map = features[0]     # (1, 512, 7, 7)

    ##### YOUR CODE START #####

    ##### YOUR CODE END #####
    
    return cam # (7, 7)

In [ ]:
model = load_pretrained_model()
image = preprocess_image("resources/airplane.jpg")
output = model(image)
class_idx = int(output.argmax())
cam = compute_cam(model, image, class_idx)
print(f"Image shape : {image.shape}, CAM shape : {cam.shape}")

assert torch.isclose(torch.sum(cam, axis = 0), torch.tensor([122.27974700927734, 169.5097198486328, 156.0562744140625, 139.6700439453125, 121.7618408203125, 130.908447265625, 95.88688659667969]), rtol=1e-1).all(), "cam activation map is different"

print("\033[92m All tests passed!")

In [ ]:
visualize_heatmap("resources/airplane.jpg", cam, imagenet_idx_to_labels[class_idx])

# GradCAM
CAM (Class Activation Map)은 Global Average Pooling 레이어와 fc 레이어가 있는 CNN 구조에서만 작동하며, 마지막 CNN 레이어의 활성화 맵만 시각화할 수 있다는 한계가이 존재합니다.

GradCAM([논문 링크](https://arxiv.org/pdf/1610.02391))은 이러한 단점을 보완하기 위해 개발되었습니다.

Grad-CAM은 임의의 CNN 레이어에 대한 클래스별 중요도를 계산하며, Transformer, RNN 등 다양한 네트워크 구조에도 확장될 수 있는 유연성을 제공합니다

## Grad-CAM 계산과정

모델의 class $c$에 대한 score값(softmax이전 logit값)을 $y^c$라 하고, 특정 conv layer의 k-번째 feature map을 $A^k$라 하겠습니다.

미분 값 $\frac{\partial y^c}{\partial A^k_{ij}}$는 $A^k$의 공간 위치 $(i, j)$가 클래스 $c$의 점수 $y^c$에 미치는 영향을 나타냅니다.

이 미분 값에 에 대해 **global average pooling**을 수행하면 중요도 $\alpha_k^c$를 계산할 수 있습니다.

$$\alpha_k^c = \overbrace{\frac{1}{Z} \sum_{i} \sum_{j}}^{\text{global average pooling}} \underbrace{\frac{\partial y^c}{\partial A^k_{ij}}}_{\text{gradients via backprop}}$$


이 $\alpha_k^c$값은 feature map $A^k$가 클래스 $c$ 예측에 미치는 중요도를 나타냅니다.

Grad-CAM $L^c_{\text{Grad-CAM}}$는 다음과 같이 얻을 수 있습니다 (class-discriminative localization map이라 불림).

$$ L^c_{\text{Grad-CAM}} = \text{ReLU} \underbrace{(\sum_{k} \alpha_k^c A^k )}_{\text{linear combination}}$$

여기서 ReLU를 사용하는 이유는 클래스 $c$에 대해 positive한 영향을 주는 부분만을 시각화하기 위해서입니다.

<img src="resources/GradCAM.png" style="width:1000px">

---

**참고** Grad-CAM은 CAM의 일반화임을 수학적으로 증명할 수 있다.

CAM 계산은 아래와 같이 주어집니다:
$$M_c(x, y) = \sum_k w_k^c A_k(x, y)$$

여기서 fc레이어의 가중치 $w_k^c$는 사실상 Grad-CAM에서 사용되는 $\alpha_k^c$와 동등합니다. 
$$ w_k^c = \sum_{i} \sum_{j} \frac{\partial y^c}{\partial A^k_{ij}} $$

자세한 유도는 Grad-CAM 논문에서 확인할 수 있습니다.

## <mark>실습</mark> GradCAM
`GradCAM`의 `__call__`을 완성하세요.


In [ ]:
class GradCAM:
    def __init__(self, model):
        self.model = model.eval()
        self.gradient = None
        self.feature_map = None
        self._register_hook()

    def _register_hook(self):
        """
        Register hooks to capture gradients and activations from the target layer.
        """
        target_layer = self.model.layer4[-1].conv2
        target_layer.register_forward_hook(self._forward_hook)
        target_layer.register_full_backward_hook(self._backward_hook)

    def _forward_hook(self, module, input, output):
        """
        Forward hook to capture activations.

        Args:
            module (torch.nn.Module): The module being hooked.
            input (torch.Tensor): Input to the module.
            output (torch.Tensor): Output from the module.
        """
        self.feature_map = output   #[1, 512, 7, 7]

    def _backward_hook(self, module, grad_input, grad_output):
        """
        Backward hook to capture gradients.

        Args:
            module (torch.nn.Module): The module being hooked.
            grad_input (tuple): Gradients with respect to the module's inputs.
            grad_output (tuple): Gradients with respect to the module's outputs.
        """
        self.gradient = grad_output[0]  #[1, 512, 7, 7]
        
    def __call__(self, x, target_class):
        """
        Compute the Grad-CAM heatmap.

        Args:
            x (torch.Tensor): Input image tensor.
            target_class (int): Target class index.

        Returns:
            torch.Tensor: Grad-CAM heatmap.
        """

        ##### YOUR CODE START #####
        output = self.model(x) # save activation map to self.feature_map

        self.model.zero_grad()
        loss = None # TODO
        loss.backward() # save gradient to self.gradient

        a_k = None # TODO, Output shape: [1, C, 1, 1]
        ##### YOUR CODE END #####

        grad_cam = torch.sum(a_k * self.feature_map, dim=1) # Output shape: [1, H, W]
        grad_cam = torch.relu(grad_cam)

        return grad_cam.squeeze()


In [ ]:
image_files = [
    "resources/airplane.jpg", 
    "resources/bus.jpg",
    "resources/dog.jpg", 
    "resources/african_hunting_dog.jpg",
    "resources/dog_cat.jpg"
]

model = load_pretrained_model()
gradcam = GradCAM(model=model)

gradcam_results = []
for image_file in image_files:
    image = preprocess_image(image_file)

    output = model(image)
    class_idx = int(output.argmax())

    gradcam_heatmap = gradcam(image, class_idx)

    # print(f"Grad-CAM shape : {gradcam_heatmap.shape}")
    visualize_heatmap(image_file, gradcam_heatmap, imagenet_idx_to_labels[class_idx])
    gradcam_results.append(gradcam_heatmap)

In [ ]:
gradcam_all = torch.stack(gradcam_results, dim = 0)

assert torch.isclose(gradcam_all[:, 3, 4], torch.tensor([0.4690302014350891, 0.600059449672699, 0.35050174593925476, 0.6419365406036377, 0.15902754664421082]), atol = 1e-2).all()
assert torch.isclose(gradcam_all[:, 1, 2], torch.tensor([0.0404171422123909, 0.41235652565956116, 0.5197665691375732, 0.07737737894058228, 0.6306908130645752]), atol = 1e-2).all()

print("\033[92m All tests passed!")

같은 이미지에서 서로 다른 클래스 $c$에 대한 Grad-CAM을 시각화해봅시다.

`gradcam_single_image`함수는 주어진 이미지 파일에 대해 Grad-CAM heatmap을 계산하고 시각화합니다.

이때, 이 함수에 `target_class` 인자를 전달하면 model의 예측값이 아니라 전달받은 `target_class`에 대한 Grad-CAM을 계산합니다.

In [ ]:
def gradcam_single_image(image_file, model, gradcam, target_class=None):
    """
    Process a single image file to compute and display the Grad-CAM heatmap.

    Args:
        image_file (str): Path to the image file.
        target_class (int, optional): Target class index. If None, use model's prediction.

    Returns:
        torch.Tensor: Grad-CAM heatmap.
    """
    image = preprocess_image(image_file)

    if target_class is None:
        output = model(image)
        class_idx = int(output.argmax())
    else:
        class_idx = target_class

    gradcam_heatmap = gradcam(image, class_idx)

    visualize_heatmap(image_file, gradcam_heatmap, imagenet_idx_to_labels[class_idx])

    return gradcam_heatmap


In [ ]:
image_file = "resources/dog_cat.jpg"

target_classes = [None, 281]

model = load_pretrained_model()
gradcam = GradCAM(model=model)

gradcam_results = []
for target_class in target_classes:
    gradcam_heatmap = gradcam_single_image(image_file, model, gradcam, target_class)

(선택 과제) 다른 마지막 CNN layer (layer4[-1].conv2) 가 아닌 다른 중간 CNN layer에서도 Grad-CAM을 시각화해보세요.

이를 통해 네트워크가 점진적으로 추출하는 특성들이 어떻게 변화하는지 확인할 수 있습니다. 초기 레이어일수록 저수준의 특징(에지나 텍스처 등)을 강조하고, 마지막 레이어일수록 고수준의 의미론적 정보(객체 형태 등)에 집중하는 경향을 보입니다. 따라서 중간 레이어에서의 Grad-CAM 시각화를 통해, 네트워크가 특정 클래스와 관련된 특징을 점진적으로 학습해가는 과정을 시각적으로 분석할 수 있습니다.

Tip: 중간 레이어의 Grad-CAM 결과를 마지막 레이어의 결과와 비교하여, 네트워크가 다양한 수준의 정보를 학습하고 사용하는 방식을 더욱 깊이 이해할 수 있습니다.

# Transformer Attention Visualization

이번에는 ViT (Vision Transformer)모델의 Attention Map을 시각화 해보겠습니다.

사전 학습된 Vision Transformer (ViT) 모델을 불러와, 입력 이미지에 대한 각 레이어에서의 attention 패턴을 시각화해봅니다. ViT는 이미지를 패치(patch) 단위로 분할하여, 각 패치 간의 관계를 Transformer의 self-attention 메커니즘으로 학습합니다. 이 실습을 통해, ViT가 입력 이미지의 특정 영역에 어떻게 주목하는지 확인할 수 있습니다.

In [ ]:
import torch
from transformers import ViTImageProcessor , ViTForImageClassification
from PIL import Image

from utils import visualize_heatmap

In [ ]:
def load_pretrained_vit():
    """
    Load a pretrained Vision Transformer model and feature extractor.

    Returns:
        model (ViTForImageClassification): Pretrained ViT model.
        feature_extractor (ViTFeatureExtractor): Corresponding feature extractor.
    """
    model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224', attn_implementation="eager")
    image_processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
    model.eval()
    return model, image_processor

def preprocess_image_vit(image_path, image_processor):
    """
    Preprocess the input image for the Vision Transformer.

    Args:
        image_path (str): Path to the input image.
        feature_extractor (ViTFeatureExtractor): Feature extractor for preprocessing.

    Returns:
        torch.Tensor: Preprocessed image tensor.
    """
    image = Image.open(image_path).convert('RGB')
    inputs = image_processor(images=image, return_tensors="pt")
    return inputs


In [ ]:
image_path = 'resources/airplane.jpg'  # Replace with your image path
model, image_processor = load_pretrained_vit()
inputs = preprocess_image_vit(image_path, image_processor)

outputs = model(**inputs, output_attentions=True)
print(f"Input shape : {inputs['pixel_values'].shape}, output shape : {outputs.logits.shape}")
print(f"Num layer : {len(outputs.attentions)}, attention shape : {outputs.attentions[0].shape}")

HuggingFace `transformer`패키지의 `ViTForImageClassification`모델은 `output_attentions=True`옵션을 통해 attention을 출력하는 기능을 제공합니다.

이때 출력되는 `outputs.attentions` 텐서는 `(batch_size, num_heads, num_tokens, num_tokens)`의 차원을 가지며 각 차원의 의미는 다음과 같습니다:
- batch_size: 배치에 포함된 이미지 수.
- num_heads: 멀티 헤드 self-attention의 헤드 수. 각 헤드는 독립적으로 attention을 계산합니다.
- num_tokens (query tokens): 트랜스포머 모델의 입력 토큰 수로, 이미지 패치와 CLS 토큰이 포함됩니다 (197 = 1 + 14*14). CLS 토큰은 분류모델에서 전체 이미지의 요약 정보로 사용됩니다.
- num_tokens (key tokens): 각 query token이 주목하는 모든 key token의 attention 값을 나타냅니다.

여기서 CLS 토큰과 각 이미지 패치간의 attention 값을 분석하면 ViT 이미지 분류 모델이 이미지의 어느 영역에 가장 집중하고 있는지 파악할 수 있습니다.

## <mark>실습</mark> Attention map visualization
아래 과정에 따라 `obtain_attention_maps`함수를 완성하세요.
1. Head Fusion: 여러 헤드의 attention 값을 평균을 통해 합쳐서 단일 attention map을 생성합니다.
2. CLS 토큰의 attention 값 추출: query가 CLS 토큰이고 key가 이미지 패치들인 attention 값들을 추출합니다.
3. 2D Attention Map으로 변환: 이미지 패치들은 1D로 나열되어 있으므로, `num_grid x num_grid` 크기로 변환합니다.

In [ ]:
def obtain_attention_maps(attentions, patch_size=16):
    """
    Obtain attention maps from the ViT model.

    Args:
        attentions (list of torch.Tensor): Attention maps from each layer.
        patch_size (int): Size of the patches used in ViT.

    Returns:
        list of torch.Tensor: List of attention maps for each layer.
    """
    num_grid = 224 // patch_size
    
    attention_maps = []
    for attention in attentions:  # attention shape: (batch_size, num_heads, num_tokens, num_tokens)
        ##### YOUR CODE START #####
        attention_heads_fused = None # TODO, Output shape:(batch_size, num_tokens, num_tokens)

        # attention from the CLS token to the image patches
        cls_attentions = None # TODO, Output shape:(num_tokens - 1)
        ##### YOUR CODE END #####
        cls_attentions = cls_attentions.reshape(num_grid, num_grid)
        attention_maps.append(cls_attentions)

    return attention_maps

def get_attention_maps(model, inputs):
    """"
    Get the attention maps from the Vision Transformer.

    Args:
        model (ViTForImageClassification): Pretrained ViT model.
        inputs (dict): Dictionary of inputs for the model (e.g., pixel_values).

    Returns:
        list of torch.Tensor: List containing attention maps from each layer.
    """
    with torch.no_grad():
        outputs = model(**inputs, output_attentions=True)
    return outputs.attentions

In [ ]:
attentions = get_attention_maps(model, inputs)
heatmaps = obtain_attention_maps(attentions)

assert torch.isclose(heatmaps[4].sum(axis = 0), torch.tensor([0.06968411803245544, 0.08526063710451126, 0.09705086797475815, 0.07389257848262787, 0.060506727546453476, 0.05866185575723648, 0.06139916554093361, 0.0615709163248539, 0.05205957964062691, 0.05494638532400131, 0.05141854286193848, 0.051894575357437134, 0.05805109441280365, 0.062026496976614]), atol = 1e-2).all(), "Attrention map value is different"

print("\033[92m All tests passed!")

In [ ]:
for i, heatmap in enumerate(heatmaps):
    visualize_heatmap(image_path, heatmap, f"Layer {i}")

## Self-Attention과 정보 혼합 (Information Mixing)
Transformer의 self-attention 레이어에서는 Attention에 기반하여 각 패치의 임베딩(embedding)이 다른 패치의 임베딩과 혼합됩니다. 트랜스포머 layer가 깊어질수록 이러한 정보 혼합이 반복되어, 각 패치가 다른 패치의 정보를 점점 더 많이 포함하게 되며, 이를 통해 각 패치가 이미지 전체의 정보를 담을 수 있게 됩니다.

Vision Transformer에서 [CLS] 토큰은 모든 패치의 정보를 종합하여 최종 분류에 사용됩니다. 각 레이어에서 [CLS] 토큰은 모든 패치에 주목하며, 레이어를 거듭할수록 이미지의 전체적인 정보가 [CLS] 토큰에 축적됩니다.

### Attention map 시각화 결과 해석
- 초기 레이어: 위 Attention map을 출력 결과를 살펴보면 초기 레이어의 attention map은 주로 이미지의 일부 영역(특정 패치)에 집중하는 경향을 보입니다.

- 깊은 레이어
  - 레이어가 깊어질수록 모델은 더 높은 수준의 추상화된 특징을 학습하게 됩니다. 이에 따라 attention map은 점점 더 이미지의 전반적인 객체나 전체적인 맥락을 반영하고, 특정 세부 영역에 대한 Attention은 줄어듭니다. 
  - 따라서 최종 레이어에 가까워질수록 [CLS] 토큰의 attention은 이미지의 특정 부분이 아닌 이미지 전체에 대한 집중도가 높아집니다. 
  - 또한 Self-attention으로 인한 정보 혼합으로 인해 깊은 레이어에서는 각 패치가 더 이상 고유한 공간적 정보를 많이 유지하지 않게 됩니다. 대신, 각 패치는 의미론적으로 풍부하지만 공간적으로 혼합된 정보를 표현합니다. 그 결과, 깊은 레이어에서의 attention map은 이미지 전체를 덮거나 특정 객체와 무관한 영역을 포함할 수 있습니다


# Attention Rollout

앞서 살펴본 ViT모델의 Attention map은 transformer layer를 거치면서 점차 양상이 달라집니다. 그러면 ViT의 전체 레이어의 종합적인 attention map은 어떻게 구할 수 있을까요?

Attention Rollout 기법은 Quantifying Attention Flow in Transformers논문([링크](https://arxiv.org/pdf/2005.00928))에서 제안된 방법으로, 각 레이어의 attention을 결합하여 Transformer 모델의 전체적인 attention 흐름을 분석하는 방법입니다.

1. Head Fusion:여러 attention 헤드가 존재하므로, 이 헤드들 간의 attention을 결합합니다. 평균(mean), 최대값(max), 최소값(min) 등 다양한 방식이 있으며 아래 수식은 평균을 사용했을때의 수식입니다:
$$A_l^{\text{avg}} = \frac{1}{N_{\text{heads}}} \sum_{h=1}^{N_{\text{heads}}} A_l^{(h)}$$

2. Residual Connection 반영: Self-attention은 일반적으로 residual connection과 함께 사용되므로 이를 반영하기 위해 identity 행렬을 더해줍니다.
$$A_l^{\text{rollout}} = A_l^{\text{avg}} + I$$

3. 정규화: 각 행의 합이 1이 되도록로 정규화합니다. 이를 통해 각 토큰이 다른 이미지 토큰에 얼마나 집중하는지를 더 명확히 파악할 수 있습니다.
$$A_l^{\text{norm}} = \frac{A_l^{\text{rollout}}}{\sum_{j=1}^{N_{\text{tokens}}} A_l^{\text{rollout}}[i, j]}$$

4. 레이어별 Attention 곱하기: 정규화된 attention 행렬들을 첫 번째 레이어부터 마지막 레이어까지 순차적으로 곱하여 전체 attention rollout $R_L$을 계산합니다.
$$R_L = A_1^{\text{norm}} \cdot A_2^{\text{norm}} \cdot \dots \cdot A_L^{\text{norm}}$$

5. CLS 토큰의 Attention: $R_L$에서 [CLS] 토큰이 이미지 패치에 대해 가지는 attention을 추출합니다. 이를 통해 CLS 토큰이 이미지 전체 정보를 어떻게 통합했는지 확인할 수 있습니다.\

$$\text{Attn}_{\text{CLS}} = R_L[0, 1:]$$


In [ ]:
def compute_attention_rollout(attentions, discard_ratio=0.0, head_fusion='mean'):
    """
    Compute the attention rollout from the attention maps.

    Args:
        attentions (list of torch.Tensor): Attention maps from each layer.
        discard_ratio (float): Ratio of attention to discard.
        head_fusion (str): Method to fuse attention heads.

    Returns:
        numpy.ndarray: Attention rollout map.
    """
    result = torch.eye(attentions[0].size(-1))  #shape: (197, 197)
    with torch.no_grad():
        for attention in attentions: # shape: (batch_size, num_heads, num_tokens, num_tokens)
            if head_fusion == "mean":
                attention_heads_fused = attention.mean(axis=1) # shape: (batch_size, num_tokens, num_tokens)
            elif head_fusion == "max":
                attention_heads_fused = attention.max(axis=1)[0]
            elif head_fusion == "min":
                attention_heads_fused = attention.min(axis=1)[0]
            else:
                raise "Attention head fusion type Not supported"

            # Remove percentages of the least important attentions
            flat = attention_heads_fused.view(attention_heads_fused.size(0), -1)
            _, indices = flat.topk(int(flat.size(-1) * discard_ratio), -1, False)
            indices = indices[indices != 0]
            flat[0, indices] = 0

            # Add identity matrix and normalize rows
            I = torch.eye(attention_heads_fused.size(-1)) # shape: (197, 197)
            a = attention_heads_fused + I
            a = a / a.sum(dim=-1)

            result = torch.matmul(a, result) #(1, 197, 197)

    mask = result[0, 0, 1:] # attention between the class token and the image patches
    num_grid = int(mask.size(-1)**0.5)
    mask = mask.reshape(num_grid, num_grid)
    return mask

In [ ]:
image_path = 'resources/dog_cat2.jpg'  # Replace with your image path
model, image_processor = load_pretrained_vit()
inputs = preprocess_image_vit(image_path, image_processor)
attentions = get_attention_maps(model, inputs)

attention_rollout = compute_attention_rollout(attentions, discard_ratio= 0.6, head_fusion= "mean")
print(f"attention_rollout shape: {attention_rollout.shape}")
visualize_heatmap(image_path, attention_rollout, 0)


In [ ]:
assert torch.isclose(attention_rollout.sum(axis = -1), torch.tensor([0.06629233062267303, 0.06181266903877258, 0.05839885398745537, 0.054853130131959915, 0.059523843228816986, 0.06684955954551697, 0.06055447831749916, 0.05917668342590332, 0.05544937774538994, 0.06043049693107605, 0.05814819782972336, 0.06188401207327843, 0.06259924918413162, 0.0535668320953846]), atol=1e-3).all()

print("\033[92m All tests passed!")